In [1]:
%load_ext lab_black

# 목차
## 0. Library
## 1. Loading Data
## 2. Feature Engineering
- Pivot table
- Find Best Model
- Scaling

## 3. Modeling
- Bayesian Optimization

## 4. Make Submission

# 0. Library

In [2]:
## 1. Loading Data
import pandas as pd

path = 'data/'

## 2. Feature Engineering

### Find Best Model with Pycaret
from pycaret.classification import *

### Scaling
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score
from lightgbm import LGBMClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler

## 3. Modeling
from sklearn.metrics import log_loss
import time
from bayes_opt import BayesianOptimization

## 4. Make Submission
from datetime import datetime
import os

ModuleNotFoundError: No module named 'pycaret'

## 1. Loading Data

In [2]:
X_train = pd.read_csv(path + 'train_features.csv')
y_train = pd.read_csv(path + 'train_labels.csv')
X_test = pd.read_csv(path + 'test_features.csv')
display(X_train.head(3))
display(y_train.head(3))
display(X_test.head(3))

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629


,id,label,label_desc
0,0,37,Shoulder Press (dumbbell)
1,1,26,Non-Exercise
2,2,3,Biceps Curl (band)


,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,3125,0,-0.628100,-0.160155,0.151487,49.665357,88.435961,13.597668
1,3125,1,-0.462548,0.012462,-0.053726,56.953059,96.185341,16.278458
2,3125,2,-0.363481,-0.091789,-0.130004,29.557396,93.836453,13.329043


## 2. Feature Engineering

### 1) Make Pivot Table
현재 데이터가 3125명 * 600개의 log 로 구성되어 있습니다.

Machine Learning Model에 넣기 위해서는 데이터를 집약해서 한명의 id에 데이터를 집약적으로 넣어야 합니다.

따라서 첫번째로 Pandas에서 지원하는 Pivot Table Method를 이용합니다.

In [3]:
X_pivot_train = pd.pivot_table(data = X_train, # X_train의 데이터를 통해서
                               values = X_train.columns[2:],  # id와 time을 제외한 피쳐를 대상으로
                               index = 'id', # id를 기준으로 잡아
                               aggfunc = ['sum','mean',         # 합, 평균
                                          'median','min','max', # 중앙값 최소값, 최대값
                                          'std','var'           # 베셀 보정 표본 표준편차, 비편향 편차 의 값을 구합니다.
                                         ]
                              )

X_pivot_test = pd.pivot_table(data = X_test, #
                               values = X_test.columns[2:], 
                               index = 'id', # id를 기준으로 잡아
                               aggfunc = ['sum','mean',        
                                          'median','min','max',
                                          'std','var'          
                                         ]
                              )

display(X_pivot_train.head(3))
display(X_pivot_test.head(3))

sum                                                                 \
         acc_x       acc_y       acc_z         gy_x         gy_y         gy_z   
id                                                                              
0   558.797337 -131.082711 -222.252919 -1119.161589 -2015.703683   709.264425   
1  -459.948117 -190.354639   -2.534051  6642.960123  1044.284884   835.976169   
2    23.901616  -49.441742  375.607013 -5083.770868   358.725917  1831.974458   

        mean                                 ...       std             \
       acc_x     acc_y     acc_z       gy_x  ...     acc_z       gy_x   
id                                           ...                        
0   0.931329 -0.218471 -0.370422  -1.865269  ...  0.135131  13.284216   
1  -0.766580 -0.317258 -0.004223  11.071600  ...  0.499395  79.244561   
2   0.039836 -0.082403  0.626012  -8.472951  ...  0.248807  25.422926   

                                var                                   \
          gy_y       gy_z     acc_x     acc_y     acc_z         gy_x   
id                                                                     
0    24.300479  25.275185  0.036664  0.031375  0.018260   176.470384   
1    96.005289  75.545343  0.245548  0.113175  0.249396  6279.700472   
2   118.956646  13.920337  0.506904  0.021646  0.061905   646.325142   

                               
            gy_y         gy_z  
id                             
0     590.513292   638.834979  
1    9217.015511  5707.098884  
2   14150.683677   193.775778  

[3 rows x 42 columns]

sum                                                     \
           acc_x       acc_y       acc_z          gy_x         gy_y   
id                                                                    
3125 -611.238360  -11.744605 -139.355669  -1911.076959  1639.123438   
3126 -313.705824  367.296809  -42.655405 -10644.915365  4184.863263   
3127  304.167948  542.291164  -84.658968  -1307.846921 -1350.871152   

                       mean                                 ...       std  \
             gy_z     acc_x     acc_y     acc_z       gy_x  ...     acc_z   
id                                                          ...             
3125 -1200.410049 -1.018731 -0.019574 -0.232259  -3.185128  ...  0.174672   
3126 -2162.747150 -0.522843  0.612161 -0.071092 -17.741526  ...  0.383800   
3127  -235.904841  0.506947  0.903819 -0.141098  -2.179745  ...  0.152077   

                                            var                      \
           gy_x       gy_y       gy_z     acc_x     acc_y     acc_z   
id                                                                    
3125  12.897967  31.993022  12.251648  0.055806  0.008398  0.030510   
3126  51.625096  45.706311  61.604867  0.291264  0.110899  0.147302   
3127  22.770845  13.467885  23.041463  0.048371  0.036666  0.023128   

                                             
             gy_x         gy_y         gy_z  
id                                           
3125   166.357553  1023.553453   150.102867  
3126  2665.150566  2089.066820  3795.159662  
3127   518.511372   181.383940   530.909012  

[3 rows x 42 columns]

#### Reset Column Name
- 현재 Column 명이 Multi index로 되어있어, 모델에 넣을 때 에러가 발생합니다.
- Column명의 변환 작업 후 id를 다시 column에 추가합니다.

In [4]:
X_pivot_train.columns[:5]

MultiIndex([('sum', 'acc_x'),
            ('sum', 'acc_y'),
            ('sum', 'acc_z'),
            ('sum',  'gy_x'),
            ('sum',  'gy_y')],
           )

In [5]:
X_columns = [agg + '_' + column for agg,column in X_pivot_train.columns]
X_pivot_train.columns = X_columns
X_pivot_test.columns = X_columns
display(X_pivot_train.head())

,sum_acc_x,sum_acc_y,sum_acc_z,sum_gy_x,sum_gy_y,sum_gy_z,mean_acc_x,mean_acc_y,mean_acc_z,mean_gy_x,...,std_acc_z,std_gy_x,std_gy_y,std_gy_z,var_acc_x,var_acc_y,var_acc_z,var_gy_x,var_gy_y,var_gy_z
id,,,,,,,,,,,,,,,,,,,,,
0,558.797337,-131.082711,-222.252919,-1119.161589,-2015.703683,709.264425,0.931329,-0.218471,-0.370422,-1.865269,...,0.135131,13.284216,24.300479,25.275185,0.036664,0.031375,0.018260,176.470384,590.513292,638.834979
1,-459.948117,-190.354639,-2.534051,6642.960123,1044.284884,835.976169,-0.766580,-0.317258,-0.004223,11.071600,...,0.499395,79.244561,96.005289,75.545343,0.245548,0.113175,0.249396,6279.700472,9217.015511,5707.098884
2,23.901616,-49.441742,375.607013,-5083.770868,358.725917,1831.974458,0.039836,-0.082403,0.626012,-8.472951,...,0.248807,25.422926,118.956646,13.920337,0.506904,0.021646,0.061905,646.325142,14150.683677,193.775778
3,-532.621192,-52.600737,136.413976,10646.500409,2880.558352,-3521.938833,-0.887702,-0.087668,0.227357,17.744167,...,0.205882,42.928860,36.953466,23.647153,0.017134,0.037639,0.042387,1842.887012,1365.558625,559.187841
4,-395.410844,-202.240064,121.654507,-2891.782899,5791.027696,2672.029417,-0.659018,-0.337067,0.202758,-4.819638,...,0.389646,108.258866,60.514531,46.148326,0.245193,0.325247,0.151824,11719.982095,3662.008463,2129.668017


In [6]:
X_pivot_train = X_pivot_train.reset_index()
X_pivot_test = X_pivot_test.reset_index()

display(X_pivot_train.head())
display(X_pivot_test.head())

,id,sum_acc_x,sum_acc_y,sum_acc_z,sum_gy_x,sum_gy_y,sum_gy_z,mean_acc_x,mean_acc_y,mean_acc_z,...,std_acc_z,std_gy_x,std_gy_y,std_gy_z,var_acc_x,var_acc_y,var_acc_z,var_gy_x,var_gy_y,var_gy_z
0,0,558.797337,-131.082711,-222.252919,-1119.161589,-2015.703683,709.264425,0.931329,-0.218471,-0.370422,...,0.135131,13.284216,24.300479,25.275185,0.036664,0.031375,0.018260,176.470384,590.513292,638.834979
1,1,-459.948117,-190.354639,-2.534051,6642.960123,1044.284884,835.976169,-0.766580,-0.317258,-0.004223,...,0.499395,79.244561,96.005289,75.545343,0.245548,0.113175,0.249396,6279.700472,9217.015511,5707.098884
2,2,23.901616,-49.441742,375.607013,-5083.770868,358.725917,1831.974458,0.039836,-0.082403,0.626012,...,0.248807,25.422926,118.956646,13.920337,0.506904,0.021646,0.061905,646.325142,14150.683677,193.775778
3,3,-532.621192,-52.600737,136.413976,10646.500409,2880.558352,-3521.938833,-0.887702,-0.087668,0.227357,...,0.205882,42.928860,36.953466,23.647153,0.017134,0.037639,0.042387,1842.887012,1365.558625,559.187841
4,4,-395.410844,-202.240064,121.654507,-2891.782899,5791.027696,2672.029417,-0.659018,-0.337067,0.202758,...,0.389646,108.258866,60.514531,46.148326,0.245193,0.325247,0.151824,11719.982095,3662.008463,2129.668017


,id,sum_acc_x,sum_acc_y,sum_acc_z,sum_gy_x,sum_gy_y,sum_gy_z,mean_acc_x,mean_acc_y,mean_acc_z,...,std_acc_z,std_gy_x,std_gy_y,std_gy_z,var_acc_x,var_acc_y,var_acc_z,var_gy_x,var_gy_y,var_gy_z
0,3125,-611.238360,-11.744605,-139.355669,-1911.076959,1639.123438,-1200.410049,-1.018731,-0.019574,-0.232259,...,0.174672,12.897967,31.993022,12.251648,0.055806,0.008398,0.030510,166.357553,1023.553453,150.102867
1,3126,-313.705824,367.296809,-42.655405,-10644.915365,4184.863263,-2162.747150,-0.522843,0.612161,-0.071092,...,0.383800,51.625096,45.706311,61.604867,0.291264,0.110899,0.147302,2665.150566,2089.066820,3795.159662
2,3127,304.167948,542.291164,-84.658968,-1307.846921,-1350.871152,-235.904841,0.506947,0.903819,-0.141098,...,0.152077,22.770845,13.467885,23.041463,0.048371,0.036666,0.023128,518.511372,181.383940,530.909012
3,3128,-346.561617,-366.333946,18.891749,485.147442,-1790.981310,-14.590798,-0.577603,-0.610557,0.031486,...,0.326569,42.818157,45.069932,37.967372,0.186376,0.054569,0.106647,1833.394532,2031.298793,1441.521366
4,3129,-443.184021,109.521180,240.781103,-858.922755,865.419381,3447.298941,-0.738640,0.182535,0.401302,...,0.261848,92.301963,67.911174,43.353007,0.093512,0.098781,0.068565,8519.652350,4611.927587,1879.483194


### 2) FInd Appropriate Model by Pycaret
- lable 데이터를 id에 맞게 merge
- pycaret 모델에 넣어 가장 성능이 좋은 model을 선택
- 해당 모델을 기반으로 모델링 진행

In [7]:
y_train.head()

,id,label,label_desc
0,0,37,Shoulder Press (dumbbell)
1,1,26,Non-Exercise
2,2,3,Biceps Curl (band)
3,3,26,Non-Exercise
4,4,26,Non-Exercise


In [8]:
train_data = pd.merge(X_pivot_train, y_train.loc[:,['id','label']], on='id') # label_desc는 사용하지 않을 예정입니다.
train_data.label = train_data.label.astype('category')

In [9]:
clf = setup(data = train_data, target = 'label', silent = True)

Setup Succesfully Completed!


,Description,Value
0,session_id,8092
1,Target Type,Multiclass
2,Label Encoded,"0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38, 39: 39, 40: 40, 41: 41, 42: 42, 43: 43, 44: 44, 45: 45, 46: 46, 47: 47, 48: 48, 49: 49, 50: 50, 51: 51, 52: 52, 53: 53, 54: 54, 55: 55, 56: 56, 57: 57, 58: 58, 59: 59, 60: 60"
3,Original Data,"(3125, 44)"
4,Missing Values,False
5,Numeric Features,43
6,Categorical Features,0
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [10]:
# Gradient Boosting Classifier, CatboostClassifeir 는 시간이 오래걸려 제외

compare_models(exclude = ['gbc', 'catboost'], sort = 'Accuracy', n_select = 1, fold = 4) 

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,Light Gradient Boosting Machine,0.7412,0.0000,0.4655,0.6941,0.6991,0.6313,0.6409,5.8085
1,Extra Trees Classifier,0.7389,0.0000,0.4552,0.6837,0.6869,0.6195,0.6358,0.2203
2,Extreme Gradient Boosting,0.7366,0.0000,0.4820,0.7036,0.7042,0.6328,0.6379,4.7446
3,Random Forest Classifier,0.7124,0.0000,0.4371,0.6641,0.6690,0.5916,0.6002,0.0826
4,Linear Discriminant Analysis,0.6850,0.0000,0.4493,0.6643,0.6584,0.5735,0.5753,0.0165
5,Decision Tree Classifier,0.6026,0.0000,0.3651,0.6131,0.5993,0.4788,0.4793,0.2638
6,Ridge Classifier,0.5716,0.0000,0.1433,0.3980,0.4468,0.2864,0.3449,0.0085
7,K Neighbors Classifier,0.5661,0.0000,0.2509,0.5146,0.5265,0.3970,0.4002,0.0122
8,Logistic Regression,0.5446,0.0000,0.1502,0.4087,0.4472,0.2886,0.3143,0.3379
9,Naive Bayes,0.5377,0.0000,0.4708,0.6690,0.5703,0.4479,0.4576,0.0075


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=8092, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

### 2) Scaling

In [11]:
best_score = 0
columns = 'scaled_' + X_pivot_train.columns
for scaler in [None, MinMaxScaler(), StandardScaler()]:
    
    if scaler == None:
        scaled_train_x = X_pivot_train.copy()
        scaled_test_x = X_pivot_test.copy()
    else:
        scaled_train_x = scaler.fit_transform(X_pivot_train)
        scaled_train_x = pd.DataFrame(scaled_train_x, columns = columns).astype('float64')
        scaled_test_x = scaler.transform(X_pivot_test)
        scaled_test_x = pd.DataFrame(scaled_test_x, columns = columns).astype('float64')
        
    skf = StratifiedKFold(n_splits=3 , shuffle=True)
    model = LGBMClassifier(n_jobs = -1)
    scores = cross_val_score(model, X_pivot_train, y_train.label.values, scoring='accuracy', cv=skf, n_jobs=-1)
    
    if scaler == None : print('NON_SCALED')
    else:    
        print(scaler.__class__.__name__)
    print(f'TOTAL 최대성능: {max(scores)}\n평균성능: {np.mean(scores)}')
    print('-'*30)

    if np.mean(scores) >= best_score:
        best_score = np.mean(scores)
        best_train_x = scaled_train_x
        best_test_x = scaled_test_x
        best_scaler = scaler

if best_scaler == None:
    print('BEST SCALER IS NON_SCALED')
else:
    print('BEST SCALER : ', best_scaler.__class__.__name__)

NON_SCALED
TOTAL 최대성능: 0.7610364683301344
평균성능: 0.7574404624717977
------------------------------
MinMaxScaler
TOTAL 최대성능: 0.7543186180422264
평균성능: 0.7481615873314391
------------------------------
StandardScaler
TOTAL 최대성능: 0.760806916426513
평균성능: 0.7488038409841415
------------------------------
BEST SCALER IS NON_SCALED


In [12]:
display(best_train_x.head())

,id,sum_acc_x,sum_acc_y,sum_acc_z,sum_gy_x,sum_gy_y,sum_gy_z,mean_acc_x,mean_acc_y,mean_acc_z,...,std_acc_z,std_gy_x,std_gy_y,std_gy_z,var_acc_x,var_acc_y,var_acc_z,var_gy_x,var_gy_y,var_gy_z
0,0,558.797337,-131.082711,-222.252919,-1119.161589,-2015.703683,709.264425,0.931329,-0.218471,-0.370422,...,0.135131,13.284216,24.300479,25.275185,0.036664,0.031375,0.018260,176.470384,590.513292,638.834979
1,1,-459.948117,-190.354639,-2.534051,6642.960123,1044.284884,835.976169,-0.766580,-0.317258,-0.004223,...,0.499395,79.244561,96.005289,75.545343,0.245548,0.113175,0.249396,6279.700472,9217.015511,5707.098884
2,2,23.901616,-49.441742,375.607013,-5083.770868,358.725917,1831.974458,0.039836,-0.082403,0.626012,...,0.248807,25.422926,118.956646,13.920337,0.506904,0.021646,0.061905,646.325142,14150.683677,193.775778
3,3,-532.621192,-52.600737,136.413976,10646.500409,2880.558352,-3521.938833,-0.887702,-0.087668,0.227357,...,0.205882,42.928860,36.953466,23.647153,0.017134,0.037639,0.042387,1842.887012,1365.558625,559.187841
4,4,-395.410844,-202.240064,121.654507,-2891.782899,5791.027696,2672.029417,-0.659018,-0.337067,0.202758,...,0.389646,108.258866,60.514531,46.148326,0.245193,0.325247,0.151824,11719.982095,3662.008463,2129.668017


# 3. Modeling

## LGBM + Bayesian Optimization
### Bayesian Optimization(BOA)은 ??
#### init_points에서 지정한 값 만큼 Random Search합니다.
#### init_points에서 나온 값을 기반으로 n_iter에서 지정한 값 만큼 최적의 파라미터를 찾아갑니다.
#### 단, BOA는 target을 Max로 만드는 방법만 있습니다.
#### 반면,  우리의 metrics인 log_loss는 minimum한 값이 더 좋은 값이 됩니다.
#### 따라서 저는 100점에서 loss만큼을 빼주어 더 큰 값을 찾도록 조정했습니다.

In [16]:
lgbm_pbounds = {
    'learning_rate': (0.001,0.5), 
    'max_depth': (5,20),
    'n_estimators' : (100,300)
                }

def lgbm_opt(learning_rate,max_depth,n_estimators):
    
    params = {
    'learning_rate': learning_rate,
    'max_depth': int(round(max_depth)),
    'n_estimators' : int(round(max_depth))
            }
    
    lgbm = LGBMClassifier(**params)
    
    
    # Cross_val_score의 Metric이 "Neg_log_loss"이기 때문에, Negative한 값들이 나옵니다. 따라서 abs로 절대값을 씌워줍니다.
    # 이 값이 Target 값이 됩니다.
    score = 100-abs(cross_val_score(lgbm, best_train_x, y_train.label.values, scoring='neg_log_loss', cv=4,n_jobs=-1).mean()) 

    return score

BO_lgbm = BayesianOptimization(f = lgbm_opt, pbounds = lgbm_pbounds, random_state=1) 

# 20번의 Random Search 후 20번의 최적의 Search를 합니다.
BO_lgbm.maximize(init_points=20, n_iter=20)


# Maximize를 통해 나온 Parameter가 저장되어있습니다.
max_params = BO_lgbm.max['params']
max_params['max_depth'] = int(round(max_params['max_depth'])) # Max_depth는 int값을 받기에 int로 변환합니다.
max_params['n_estimators'] = int(round(max_params['n_estimators'])) # n_estimators는 int값을 받기에 int로 변환합니다.
print(max_params)

# Model에 저장된 param을 지정해 저장후, 재 학습을 통해 predict합니다.
model_lgbm = LGBMClassifier(**max_params)
model_lgbm.fit(best_train_x,  y_train.label.values, eval_metric = 'logloss')
prediction = model_lgbm.predict_proba(best_test_x)

|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------
|  1        |  75.46    |  0.2091   |  15.8     |  100.0    |
|  2        |  93.09    |  0.1519   |  7.201    |  118.5    |
|  3        |  98.56    |  0.09394  |  10.18    |  179.4    |
|  4        |  73.63    |  0.2699   |  11.29    |  237.0    |
|  5        |  98.64    |  0.103    |  18.17    |  105.5    |
|  6        |  71.99    |  0.3356   |  11.26    |  211.7    |
|  7        |  98.51    |  0.07105  |  7.972    |  260.1    |
|  8        |  80.28    |  0.4842   |  9.701    |  238.5    |
|  9        |  73.19    |  0.4383   |  18.42    |  117.0    |
|  10       |  98.06    |  0.02049  |  7.547    |  275.6    |
|  11       |  98.52    |  0.05008  |  11.32    |  291.6    |
|  12       |  79.42    |  0.267    |  15.38    |  163.1    |
|  13       |  73.41    |  0.3436   |  17.52    |  103.7    |
|  14       |  79.19    |  0.3753   |  19.83    |  249.6    |
|  15   

# 5. Make Submission
- Score에서 나온 144는 loss가 1.44라는 뜻입니다.

In [21]:
os.mkdir(path+'submission')

sample_submission = pd.read_csv(path + 'sample_submission.csv')
sample_submission.iloc[:,1:]=prediction
score = abs(cross_val_score(model_lgbm, best_train_x, y_train.label.values, scoring='neg_log_loss', cv=4, n_jobs=-1).mean())
score = str(round(score,2)).replace('.','')
print(score)

month = str(datetime.now().date().month).zfill(2)
day = str(datetime.now().date().day).zfill(2)
date = month+day

sample_submission.to_csv(path+f'submission/{date}_{best_scaler}_{model_lgbm.__class__.__name__}_{score}.csv',index=False)

144
